In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import get_dimensions
from model import *
from loss import *
from train import *
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 6
BATCH_SIZE = 1

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device('cuda')
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = pyramid_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = pyramid_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
# loaders come with auto batch division and multi-thread acceleration

In [4]:
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

def init_weights(m):
    if isinstance(m, nn.Conv3d):
        init.kaiming_normal_(m.weight.data)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=0, std=1)

def downsample_label(label, scale_factor):
    return F.interpolate(label, scale_factor=scale_factor, mode='trilinear', align_corners=True)

In [ ]:
def shape_test(model, cuda_bool):
    x = torch.zeros((1, 1, 256, 256, 256))
    x = x.to(device=device, dtype=dtype) if cuda_bool else x
    scores = model(x)
    for i in scores:
        print(i.size())

In [ ]:
#icnet1 = FullResolutionICNet(num_classes=3)
#icnet1 = nn.DataParallel(icnet1)
#icnet1 = convert_model(icnet1)
##icnet1.apply(init_weights)
#icnet1 = icnet1.to(device=device, dtype=dtype)
#shape_test(icnet1, True)
# create the model, by default model type is float, use model.double(), model.float() to convert
# move the model to desirable device

#optimizer1 = optim.Adam(icnet1.parameters(), lr=1e-2)
# create an optimizer object
# note that only the model_2 params and model_4 params will be optimized by optimizer

In [ ]:
icnet1 = FullResolutionICNet(num_classes=3)
icnet1 = nn.DataParallel(icnet1)
icnet1 = convert_model(icnet1)
optimizer1 = optim.Adam(icnet1.parameters(), lr=1e-2)

checkpoint = torch.load('../full_res_save/2019-07-28 21:54:09.866803.pth')

icnet1.load_state_dict(checkpoint['state_dict_1'])
icnet1 = icnet1.to(device=device, dtype=dtype)

optimizer1.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer1, patience=5)

In [ ]:
epochs = 5000

record = open('train_new_full_res_model_upsampled.txt','a')

logger = {'train':[], 'validation_1':[]}

for e in tqdm(range(epoch, epochs)):
# iter over epoches

    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
    
        icnet1.train()
        # Set the model flag to train
        # 1. enable dropout
        # 2. batchnorm behave differently in train and test
        
        image_4 = batch['image4_data'].to(device=device, dtype=dtype)
        label_4 = batch['image4_label'].to(device=device, dtype=dtype)
        
        image_2 = batch['image2_data'].to(device=device, dtype=dtype)
        label_2 = batch['image2_label'].to(device=device, dtype=dtype)
        
        image_1 = batch['image1_data'].to(device=device, dtype=dtype)
        label_1 = batch['image1_label'].to(device=device, dtype=dtype)
        # move data to device, convert dtype to desirable dtype
        
        # Downsample labels to coincide with icnet model outputs
        #label_1_resize_2 = downsample_label(label_1, 1/2)
        #label_2_resize_2 = downsample_label(label_2, 1/2)
        #label_4_resize_2 = downsample_label(label_4, 1/2)
        
        out_1, out_2, out_4 = icnet1(image_1)
        # do the inference

        #loss_4 = dice_loss_3(out_4, label_4_resize_2)
        #loss_2 = dice_loss_3(out_2, label_2_resize_2)
        loss_4 = dice_loss_3(out_4, label_4)
        loss_2 = dice_loss_3(out_2, label_2)
        loss_1 = dice_loss_3(out_1, label_1)
        # calculate loss

        loss = loss_4 + loss_2 + loss_1 
        # add loss
        
        epoch_loss += loss.item()
        # record minibatch loss to epoch loss
        
        optimizer1.zero_grad()
        # set the model parameter gradient to zero
        
        loss.backward()
        # calculate the gradient wrt loss
        
        optimizer1.step()
        #scheduler.step(loss)
        # take a gradient descent step
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if e%5 == 4:
    # do validation every 5 epoches
    
        icnet1.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
            # iter over validation mini batches
            
                #image_4_val = vbatch['image4_data'].to(device=device, dtype=dtype)
                #if get_dimensions(image_4_val) == 4:
                #    image_4_val.unsqueeze_(0)
                #label_4_val = vbatch['image4_label'].to(device=device, dtype=dtype)
                #if get_dimensions(label_4_val) == 4:
                #    label_4_val.unsqueeze_(0)
                
                #image_2_val = vbatch['image2_data'].to(device=device, dtype=dtype)
                #if get_dimensions(image_2_val) == 4:
                #    image_2_val.unsqueeze_(0)
                #label_2_val = vbatch['image2_label'].to(device=device, dtype=dtype)
                #if get_dimensions(label_2_val) == 4:
                #    label_2_val.unsqueeze_(0)
                
                image_1_val = vbatch['image1_data'].to(device=device, dtype=dtype)
                if get_dimensions(image_1_val) == 4:
                    image_1_val.unsqueeze_(0)
                label_1_val = vbatch['image1_label'].to(device=device, dtype=dtype)
                if get_dimensions(label_1_val) == 4:
                    label_1_val.unsqueeze_(0)
                
                #print("image_1_val:", image_1_val.shape)
                #print("label_1_val:", label_1_val.shape)
                # move data to device, convert dtype to desirable dtype
                # add dimension to labels if they are 4D tensors
                
                # Downsample labels to coincide with icnet model outputs
                #label_1_val_resize_2 = downsample_label(label_1_val, 1/2) 
                #label_2_val_resize_2 = downsample_label(label_2_val, 1/2) 
                #label_4_val_resize_2 = downsample_label(label_4_val, 1/2) 
                #print("label_1_val_resize:", label_1_val_resize_2.shape) 
                
                out_1_val = icnet1(image_1_val)
                # do the inference
                
                #print("out_4:", out_4_val.shape)
                #print("label_4:", label_4_val_resize_2.shape)
                #loss_4 = dice_loss_3(out_4_val, label_4_val_resize_2)
                #loss_2 = dice_loss_3(out_2_val, label_2_val_resize_2)
                loss_1 = dice_loss_3(out_1_val, label_1_val)
                # calculate loss

                #valloss_4 += loss_4.item()
                #valloss_2 += loss_2.item()
                valloss_1 += loss_1.item()
                # record mini batch loss
            
            outstr = '------- 1st valloss={0:.4f}'\
                .format(valloss_1/(v+1)) + '\n'
            
            logger['validation_1'].append(valloss_1/(v+1))
            
            print(outstr)
            record.write(outstr)
            record.flush()
            
            scheduler.step(valloss_1/(v+1))
            
            save_1('full_res_save', icnet1, optimizer1, logger, e)

record.close()

  0%|          | 0/4761 [00:00<?, ?it/s]

Epoch 239 finished ! Training Loss: 0.7247



  0%|          | 1/4761 [08:02<638:16:40, 482.73s/it]

------- 1st valloss=0.2253

Checkpoint 239 saved !


  0%|          | 2/4761 [13:38<580:01:40, 438.77s/it]

...

Epoch 392 finished ! Training Loss: 0.7380



  3%|▎         | 155/4761 [15:00:33<438:48:49, 342.97s/it]

Epoch 393 finished ! Training Loss: 0.7349

Epoch 394 finished ! Training Loss: 0.7436

------- 1st valloss=0.1993



  3%|▎         | 156/4761 [15:07:05<457:25:14, 357.59s/it]

Checkpoint 394 saved !
